# TensorFlow で学ぶ簡単な MNIST 手書き文字認識
このページのコードを次々と実行することで、TensorFlow の雰囲気がつかめます。
なお、現在(2019.4)の TensorFlow は Keras の I/F になっており、このコーディングスタイルは最新でないことに注意してください。また、使用している tensorflow.examples.tutorials.mnist も deprecated のようで、official な mnist を使うようにワーニングが出ます。

## まずは MNIST データのダウンロード

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
 
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## 次に TensorFlow の数式を入力
最後に print で y と y の type (クラス名が表示される) を表示させてます。

In [2]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

print(y, type(y))

Instructions for updating:
Colocations handled automatically by placer.
Tensor("Softmax:0", shape=(?, 10), dtype=float32) <class 'tensorflow.python.framework.ops.Tensor'>


## 学習の為の逆伝搬(Backpropagation) も設定

In [3]:
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

## Session を作成して学習
with で後処理をするようにしましょう。Google Colab とかで with なしで Session を作ると最後はメモリリソースがたりなくなり Kernel を再起動する羽目になります。
最後に学習した W と b を eval することで NumPy 形式に変換してます。

In [4]:
with tf.Session() as sess:
  tf.initialize_all_variables().run()
  for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    train_step.run({x: batch_xs, y_: batch_ys})
    
  correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  print(accuracy.eval({x: mnist.test.images, y_: mnist.test.labels}))
  
  w0 = W.eval()
  b0 = b.eval()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.9183


## 学習データを再確認
type で確認すると numpy.ndarray になっているはずです。

In [5]:
print(w0.shape)
print(b0.shape)
print(b0)
print(type(b0))

(784, 10)
(10,)
[-1.1666178   1.2174083   0.24349378 -0.6251349   0.09548604  2.452724
 -0.5015322   1.4489193  -2.636067   -0.5286423 ]
<class 'numpy.ndarray'>


## 学習データをテキストに落とす


In [6]:
with open('w_value.txt', 'w') as f:
  for i in range(28*28):
    delim = ''
    for j in range(10):
      f.write(f'{delim}{w0[i][j]}')
      delim = ', '
    f.write('\n')

In [7]:
with open('b_value.txt', 'w') as f:
  delim = ''
  for i in range(10):
    f.write(f'{delim}{b0[i]}')
    delim = ', '
  f.write('\n')

## ダウンロード(Google Colob の場合)
Google Colab なら次の方法でダウンロードできます。ただし Firefox では動作しない模様。

In [ ]:
from google.colab import files
files.download('w_value.txt')
files.download('b_value.txt')